## Create an Excel of unqiue records from Qdrant
### To generate the Excel file in this notebook's directory:
1. Choose local or cloud
2. Select the qdrant collection name 
3. Click `Run All` in the notebook

## 0. Imports and Configs


In [1]:
import os
import library_utils as lib


# Config Qdrant``
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
# QDRANT_URL = st.secrets["QDRANT_URL"]
# QDRANT_API_KEY = st.secrets["QDRANT_API_KEY"]
QDRANT_PATH = "/Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/Drews_Tools/qdrant_ASK_lib_tools/qdrant_db"


# Configure access to push changes to Github
GITHUB_ACCESS_TOKEN_ASK_LIBRARYCATALOG = os.getenv(
    "GITHUB_ACCESS_TOKEN_ASK_LIBRARYCATALOG")

## 1. Choose Local or Cloud

In [2]:
%pip install qdrant_client -q --upgrade

Note: you may need to restart the kernel to use updated packages.


In [3]:
from qdrant_client import QdrantClient

# Your choice of parameters here affects the rest of the notebook.
# If you change it here, be sure to click through the rest of the cells
# to reset the variables

client = QdrantClient(
    url=QDRANT_URL,  # for cloud
    api_key=QDRANT_API_KEY,  # for cloud
    # path=QDRANT_PATH,  # for local
)

qdrant_location = lib.which_qdrant(client)
lib.list_collections(client)

qdrant location: cloud

Available collections:
ASK_vectorstore
ask_pdf_docs
ASK_vectorstore-backup21APR2025
ask_pdf_pages


## 2. Select the Qdrant collection

In [4]:
collection_name = "ASK_vectorstore"

## 3. Create a dictionary of pdfs

In [5]:
def create_unique_docs_list():

    all_records = client.scroll(
        collection_name=collection_name,
        limit=100000,
    )

    records = all_records[0]

    unique_docs_ids = set()
    unique_docs_records = []

    # Iterate through each record in the records list
    for record in records:
        pdf_id = record.payload["metadata"]["pdf_id"]
        record.payload["metadata"].pop("page")

        # Skip the record if it has no `pdf_id` or is already processed
        if not pdf_id or pdf_id in unique_docs_ids:
            continue

        # Add the record's `pdf_id` to the set and update the dictionary
        unique_docs_ids.add(pdf_id)
        unique_docs_records.append(record.payload["metadata"])

    return unique_docs_records


unique_docs_list = create_unique_docs_list()
print(f"Number of unique records: {len(unique_docs_list)}\n")
print("Example Record:")
unique_docs_list[0]

Number of unique records: 222

Example Record:


{'publication_number': 'COMDTPUB P3120.17B',
 'source': './raw_pdfs/USCG Incident Mgmt Handbook COMDTPUB_P3120.17B_2014-05-01.pdf',
 'page_count': 382,
 'issue_date': '2014-05-22T00:00:00Z',
 'upsert_date': '2024-12-18T23:28:56Z',
 'pdf_id': '4cd5a55c-f37b-5c5d-b13d-b34d0a523051',
 'aux_specific': False,
 'scope': 'national',
 'unit': '',
 'organization': 'CG-5RI',
 'embedding': 'text-embedding-ada-002',
 'public_release': True,
 'title': 'U.S. Coast Guard Incident Management Handbook: Incident Command System (ICS), COMDTPUB P3120.17B',
 'expiration_date': '2024-05-21T00:00:00Z'}

## 4. Format the Dictionary

In [6]:
import pandas as pd


unique_docs_df = pd.DataFrame(unique_docs_list)

column_order = [
    "title", "publication_number", "organization", "page_count", "scope", "unit",
    "issue_date", "expiration_date", "upsert_date", "aux_specific",
    "public_release", "pdf_id", "embedding", "source"
]

# Add any missing columns
missing_columns = [
    col for col in unique_docs_df.columns if col not in column_order]
column_order.extend(missing_columns)

unique_docs_df = unique_docs_df[column_order]

unique_docs_df.iloc[0]

title                 U.S. Coast Guard Incident Management Handbook:...
publication_number                                   COMDTPUB P3120.17B
organization                                                     CG-5RI
page_count                                                        382.0
scope                                                          national
unit                                                                   
issue_date                                         2014-05-22T00:00:00Z
expiration_date                                    2024-05-21T00:00:00Z
upsert_date                                        2024-12-18T23:28:56Z
aux_specific                                                      False
public_release                                                     True
pdf_id                             4cd5a55c-f37b-5c5d-b13d-b34d0a523051
embedding                                        text-embedding-ada-002
source                ./raw_pdfs/USCG Incident Mgmt Handbook COM

## 5. Save as Excel
Saves locally. If qdrant cloud was used, it also saves to https://github.com/drew-wks/ASK/tree/main/docs/library_catalog

In [7]:
# %pip install PyGithub

from github import Github


def upload_to_github(file_name, github_token, repo_name, branch):
    """
    Saves file locally, and, uploads the specified file to the ASK app's GitHub repository if the Qdrant location is 'cloud'.
    """
    # Read the file content after it has been created.
    with open(file_name, "rb") as f:
        content = f.read()

    branch_file_path = f"docs/library_catalog/{file_name}"
    commit_message = f"Update {branch_file_path}"

    g = Github(github_token)
    repo = g.get_repo(repo_name)
    try:
        repo.create_file(branch_file_path, commit_message, content, branch)
        print(f"File '{branch_file_path}' saved to repo '{repo_name}'")
    except Exception as e:
        print(f"Failed to create '{branch_file_path}': {e}")

In [8]:
from datetime import datetime, timezone


now_utc = datetime.now(timezone.utc)
timestamp = now_utc.strftime('%Y-%m-%dT%H%M%SZ')


def write_docs_report_xlsx(unique_docs_df):
    # Determine file name based on qdrant_location
    if qdrant_location == "cloud":
        file_name = f'docs_report_qdrant_cloud_{timestamp}.xlsx'
    elif qdrant_location == "local":
        file_name = f'docs_report_qdrant_local_{timestamp}.xlsx'
    else:
        raise ValueError("Unexpected qdrant_location value.")

    # First, write the Excel file to disk.
    unique_docs_df.to_excel(file_name, index=False)
    print(f"Excel file created: {file_name}")

    # Then, if running in 'cloud' mode, upload the file to GitHub.
    if qdrant_location == "cloud":
        upload_to_github(
            file_name, GITHUB_ACCESS_TOKEN_ASK_LIBRARYCATALOG, "drew-wks/ASK", "main")

    return file_name


# Example usage:
file_name = write_docs_report_xlsx(unique_docs_df)
print(f"File saved locally as: '{file_name}'")

Excel file created: docs_report_qdrant_cloud_2025-04-22T163304Z.xlsx
File 'docs/library_catalog/docs_report_qdrant_cloud_2025-04-22T163304Z.xlsx' saved to repo 'drew-wks/ASK'
File saved locally as: 'docs_report_qdrant_cloud_2025-04-22T163304Z.xlsx'


### NOTE: You may need to reboot streamlit so that it recognizes the update.

### Utility to close Qdrant client

In [9]:
lib.close_qdrant(client)

### REFERENCES

#### How to access diffferent parts of a record

```python
all_records  # a tuple
records = all_records[0]  # records is a list
records[0].id  # a string
records[0].payload  # a dict
records[0].payload["metadata"]  # a dict
records[0].payload["metadata"]["pdf_id"]  # a string